## Recursive inference (multi-step) for BSCTRFM models

### use time series instead of SLDB arrays for easier and more efficient timestamp management

In [1]:
import os
import json
import numpy as np
import pandas as pd
import joblib
from datetime import datetime
from math import sqrt
from sklearn.metrics import mean_squared_error, mean_absolute_error

# uncomment the following line for compatibility with TensorFlow 1.15 (on GCP)
# import tensorflow.compat.v1 as tf
# uncomment the following line for TensorFlow 2.X (local execution)
import tensorflow as tf

# forecast model was saved in TensorFlow 1.15
# but, in order to make predictions locally, has to be loaded with TensorFlow 2
from tensorflow.saved_model import load

In [2]:
from bokeh.plotting import figure, show, output_file, save
from bokeh.io import output_notebook
# select a palette
from bokeh.palettes import d3
output_notebook()

Loading BokehJS ...

In [3]:
# symmetrical mean absolute percentage error
def smape(targets, predictions):
    '''
    predictions: a list with the predicted values
    targets: a list with the actual values
    '''
    import numpy as np
    # lists to NumPy arrays
    targets, predictions = np.array(targets), np.array(predictions)
    # verify predictions and targets have the same shape
    if predictions.shape == targets.shape:
            return(np.sum(2*np.abs(predictions - targets) /
                          (np.abs(targets) + np.abs(predictions)))/predictions.shape[0])

In [4]:
# a function to encode float values for serialized examples
def _float_feature_from_list_of_values(list_of_values):
    """Returns a float_list from a list of floats / doubles."""
    return tf.train.Feature(float_list=tf.train.FloatList(value=list_of_values))

In [5]:
# converts a set of tensors to a feature dict to a serialized example to pass it
# to the prediction function of the saved model 
def input_tensors_to_serialized_example(encoder_input_float_tensor,
                                        decoder_input_float_tensor,
                                        id_float_tensor):
    # first, pass the float tensors to NumPy array, then flatten them
    encoder_input_flat_array = encoder_input_float_tensor.numpy().flatten()
    decoder_input_flat_array = decoder_input_float_tensor.numpy().flatten()
    id_flat_array = id_float_tensor.numpy().flatten()
    
    # second, build the protobuffer example
    example = tf.train.Example(
        # features within the example
        features=tf.train.Features(
            # feature definition
            feature={
                'encoder_input': _float_feature_from_list_of_values(encoder_input_flat_array),
                'decoder_input': _float_feature_from_list_of_values(decoder_input_flat_array),
                'id': _float_feature_from_list_of_values(id_flat_array)
            }
        )
    )    
    # third, serialize the example dictionary to a string
    serialized_example = example.SerializeToString()
    # fourth, wrap the serialized example as a NumPy-string array
    numpy_example = np.array(serialized_example, dtype='S')
    # fifth, wrap the NumPy-string array as a string tensor
    serialized_example = tf.convert_to_tensor(numpy_example)

    return serialized_example

In [25]:
PROJECT_ROOT = '/home/developer/gcp/cbidmltsf'

In [26]:
# during batch prediction, the model identifier is obtained via Abseil Flags
# remember this notebook is based on local execution,
# therefore model directory must be downloaded from GS before running the notebook
model_id = 'BSCTRFM_TPU_014'

In [27]:
# during batch prediction, the SLDB identifier is obtained via Abseil Flags
# THE SLDB FOR INFERENCE MUST BE THE SAME USED FOR TRAINING! (THE ONE SETUP IN THE CONFIGURATION FILE)
sldb_id = 'LD2011-2014_MT320-MT330_BSCTRFM_168_168'

In [28]:
# during batch prediction, the dataset name is obtained via Abseil Flags
dataset = 'test'

In [29]:
# define a forecast window to guide the iterative prediction process
# start with a hourly, day-ahead process
forecast_window = 24

In [30]:
# ADD AN INFERENCE IDENTIFIER, BECAUSE FOR TRANSFORMER-BASED MODELS, DIFFERENT INFERENCES
# CAN BE PRODUCED FROM A SINGLE SAVED MODEL (USUALLY TO PRODUCE DIFFERENT FORECAST WINDOWS)
# during batch prediction, the inference identifier should be obtained via Abseil Flags
inference = '{:03d}'.format(forecast_window)

In [31]:
# build a path to the SLDB json file
data_dir = '{}/{}/{}'.format(PROJECT_ROOT, 'sldbs', sldb_id)

# then get the ts_identifier from the json file in the sldb directory
sldb_json_file = '{}/sldb.json'.format(data_dir)

In [32]:
# open the json file
with open(sldb_json_file, 'r') as inputfile:
    sldb_dict = json.load(inputfile)

In [33]:
# and get the time series identifier
ts_identifier = sldb_dict['ts']
ts_identifier

'LD2011-2014_MT320-MT330'

In [ ]:
# get the SLDB parameters for the forecasting model
config_json_file = '{}/{}/{}.json'.format(PROJECT_ROOT,
                                          'parameters',
                                          model_id)

# recover the sldb dictionary from the json file in parameters/
with open(config_json_file, 'r') as inputfile:
    configuration = json.load(inputfile)

In [ ]:
m = sldb_dict['embedding']['hourly']
m

In [ ]:
t = sldb_dict['no_targets']
t

In [ ]:
# verify the values of the variables for batch inference
model_id, dataset, inference

In [ ]:
encoder_input_columns = [
    'kw_scaled',
    'sin_hour_day',
    'cos_hour_day',
    'sin_day_week',
    'cos_day_week',
    'sin_day_month',
    'cos_day_month',
    'sin_day_year',
    'cos_day_year'
]

In [ ]:
decoder_input_columns = encoder_input_columns

In [ ]:
id_columns = ['token_id']

In [7]:
# now get the time series for the test dataset (unseen data)

In [8]:
# define a identifier string to access to the preprocessed time series
identifier = 'LD2011-2014'

In [9]:
# build the time series directory
time_series_folder = '/home/developer/gcp/cbidmltsf/timeseries/{}'.format(identifier)

In [10]:
# use a dictionary to remain the code consistent with the SLDB building process
# most of the times, only ts['test'] will be used for inference
# however, ts['eval'] might also be used, as it have not really been seen by training process
# (no tranining modification resulted from evaluation stage)

### rename the time series dictionary to ts_test, use the customer_id as key

In [11]:
ts_test = dict()

In [12]:
# a dictionary to store predictions detail dataframe per customer id
predictions_detail = dict()

In [13]:
# a dictionary to store predictions summary dataframe per customer id
predictions_summary = dict()

In [65]:
customer_id = 'MT_321'

In [66]:
# build the time series filename
ts_filename = '{}_{}'.format(customer_id, 'test')
# ts_filename

In [67]:
ts_test[customer_id] = pd.read_pickle('{}/{}.pkl'.format(time_series_folder, ts_filename))
# ts_test[customer_id]

In [68]:
# for consistency, rename the column date as timestamp
ts_test[customer_id] = ts_test[customer_id].rename(columns={"date": "timestamp"})

In [69]:
ts_test[customer_id] = ts_test[customer_id].set_index('timestamp')

In [70]:
# ts_test[customer_id]

In [71]:
# review start and end dates for test dataset, use relative-index slices instead of scalar index!!!
ts_test[customer_id].index[:1], ts_test[customer_id].index[-1:]

(DatetimeIndex(['2014-08-18 01:00:00'], dtype='datetime64[ns]', name='timestamp', freq=None),
 DatetimeIndex(['2014-09-07 23:00:00'], dtype='datetime64[ns]', name='timestamp', freq=None))

### and the time series to produce the test SLDB is now verified

In [72]:
# load the scaler for the requested customer_id
scaler_path = '{}/scalers/{}_min_max.save'.format(data_dir, customer_id)
scaler_path

'/home/developer/gcp/cbidmltsf/sldbs/LD2011-2014_MT320-MT330_BSCTRFM_168_168/scalers/MT_321_min_max.save'

In [73]:
min_max_scaler = joblib.load(scaler_path)

/home/developer/anaconda3/lib/python3.7/site-packages/sklearn/base.py:334: UserWarning: Trying to unpickle estimator MinMaxScaler from version 0.24.2 when using version 0.23.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


### run inference process and build databases

In [74]:
# during batch prediction, the execution identifier is obtained via Abseil Flags
for execution in [4]:
    # a columns list for the predictions dataframe
    pred_df_columns = ['model_id',
                       'execution',
                       'dataset',
                       'inference',
                       'customer_id',
                       'string_timestamps',
                       'predictions',
                       'targets']
    
    # build the predictions dataframe as a key-value pair of the dictionary
    predictions_detail[customer_id] = pd.DataFrame(columns=pred_df_columns)

    # use model identifier and execution number to build the model directory string
    model_dir = '{}_{:02d}'.format(model_id, execution)

    # get the path to the saved model main directory
    saved_model_path = '{}/{}/{}/export/exporter'.format(PROJECT_ROOT,
                                                         'models',
                                                         model_dir)

    # get all the files in the saved model path, to find the most recent one
    all_files = os.listdir(saved_model_path)
    # get the path to the most recent saved model
    latest_saved_model_id = sorted(all_files)[-1]

    # build the full path for the latest saved model dir
    export_dir = '{}/{}'.format(saved_model_path, latest_saved_model_id)
    print ('Exported model path is {}'.format(export_dir))

    # load the saved model and the prediction function
    imported = load(export_dir=export_dir, tags='serve')
    predict_fn = imported.signatures["serving_default"]
    
    # iterate on a set of valid rows of the test dataset
    starting_point = 0 # based on the inference dataset
    span = 1 + 6*24 # number of days in the test dataset, expressed in hours
    dataset_row_indexes_list = starting_point + np.arange(span)
    
    for start_index in dataset_row_indexes_list:

        # define first prediction interval with start- and end-index
        # given the interval time_series[start_index:end_index]
        # the conditioning range is the union of the encoder-input and the decoder-input
        # and the prediction range is only the last lecture in the interval,
        # by means of a recursive inference process
        # on each step the last prediction is added to the decoder input
        # and the prediction range grows one step into the future

        # get the end-index of this recursive inference interval
        end_index = start_index + (m + t)

        # initialize a list to store recurrent predictions for this interval
        predictions_list = list()

        for i in np.arange(forecast_window):

            # build the inference interval as a sub-series of the dataset
            sub_series = ts_test[customer_id][start_index + i : end_index + i]

            # important: build sources as copies of the sub-series (and therefore of the global time series)
            # to avoid overwriting the original dataset

            # the encoder input source
            encoder_input = sub_series[encoder_input_columns][:m].copy()

            # the decoder input source
            # decoder_input = sub_series[m-1:-1].copy()
            decoder_input = sub_series[decoder_input_columns][m-1:m-1+t].copy()
            
            # the id (integer) for the customer
            id_input = sub_series[id_columns][:1].copy()

            # on first step (i=0), the decoder input carries only true values
            # and the predictions list is empty
            # on subsequent steps, the decoder input includes all previous predictions
            # (stored in the predictions list)
            if i > 0:
                decoder_input['kw_scaled'][-i:] = predictions_list

            # the target source, for metrics calculation
            # the first part of the sub-series is the encoder input, and
            # the second part of the sub-series is the target (only the variable column!)
            target = sub_series['kw_scaled'][m:].copy()

            # build source tensors from the sub-series    
            encoder_input_tensor = tf.expand_dims(encoder_input, axis=0)
            decoder_input_tensor = tf.expand_dims(decoder_input, axis=0)
            id_tensor = tf.expand_dims(id_input, axis=0)

            # make input example for the prediction function
            input_example = input_tensors_to_serialized_example(encoder_input_tensor,
                                                                decoder_input_tensor,
                                                                id_tensor)

            # get the output of the prediction function as a dictionary
            predict_output_dict = predict_fn(input_example)

            # get the prediction output tensor
            predict_output_tensor = predict_output_dict['forecast']

            # get the most recent prediction
            most_recent_prediction = predict_output_tensor[0, :, 0].numpy()[-1]

            # append the most recent prediction timestep to the predictions list
            predictions_list.append(most_recent_prediction)

            # pass the predictions list to an array
            # current_predictions_array = np.array(predictions_list).reshape(-1, 1)
            # get the targets vector to be compared with the current predictions array
            # current_targets = np.array(target[-i-1:]).reshape(-1, 1)

            # calculate SMAPE on the rescaled variable
            # rescaled_predictions = min_max_scaler.inverse_transform(current_predictions_array)
            # rescaled_targets = min_max_scaler.inverse_transform(current_targets)

            # current_smape = smape(rescaled_targets, rescaled_predictions)
            # print('On row {}, SMAPE for the first {} rescaled prediction(s) is {}'.format(start_index,
            #                                                                               i + 1,
            #                                                                               current_smape))        

        # iterative predictions over the forecast window reside in predictions_list
        # convert list to array, then expand feature dimension with value 1
        predicted_values = np.array(predictions_list).reshape(-1, 1)

        # inverse-scale predictions
        rescaled_predicted_values = min_max_scaler.inverse_transform(predicted_values)

        # and the true values remain in the prediction tensor, pass them to a NumPy array
        # for the true values array, expand feature dimension with value 1
        true_values = np.array(target[-i-1:]).reshape(-1, 1)

        # inverse-scale true values
        rescaled_true_values = min_max_scaler.inverse_transform(true_values)

        # a temporary dataframe built from the data in the current row
        df = pd.DataFrame(columns=pred_df_columns)
        df['model_id'] = [model_id]
        df['execution'] = [execution]
        df['dataset'] = [dataset]
        df['inference'] = [inference]
        df['customer_id'] = [customer_id]
        df['string_timestamps']= [pd.to_datetime(target.index[-i-1:]).astype(str).tolist()]
        df['predictions'] = [np.squeeze(rescaled_predicted_values).tolist()]
        df['targets'] = [np.squeeze(rescaled_true_values).tolist()]
        
        # calculate mean absolute error and normalized deviation
        mae = mean_absolute_error(rescaled_true_values, rescaled_predicted_values)
        df['mae'] = mae
        
        true_values_average = np.mean(rescaled_true_values)
        df['nd'] = mae/true_values_average
        
        # calculate root mean squared error and normalized root mean squared error
        rmse = sqrt(mean_squared_error(rescaled_true_values, rescaled_predicted_values))
        df['rmse'] = rmse
        df['nrmse'] = rmse/true_values_average
        
        df['smape'] = smape(rescaled_true_values, rescaled_predicted_values)

        # append the temporary dataframe to the predictions detail dataframe
        predictions_detail[customer_id] = pd.concat([predictions_detail[customer_id], df])

    
    # reset the index of final dataframe, once all of its rows (dataset) have been processed
    predictions_detail[customer_id] = predictions_detail[customer_id].reset_index(drop=True)
    

    # build a predictions summary dataframe, reset index to avoid making a multi-column index when grouping by
    predictions_summary[customer_id] = predictions_detail[customer_id].groupby(['model_id',
                                                                                'execution',
                                                                                'dataset',
                                                                                'inference',
                                                                                'customer_id']).mean().reset_index()

    
    # DO NOT CALCULATE THE VECTOR METRICS
    calculate_vector_metrics = False
    
    if calculate_vector_metrics:
        
        # a range to iterate on prediction timesteps
        targets_range = np.arange(forecast_window)

        # vector metric (vector component to vector component)
        # an array forecast_window-d: metric for 1, 2,..., no_targets step-ahead
        # (target versus prediction for rows in dataset)

        # for index, row in dataframe.iterrows()
        mae_vector = [
            mean_absolute_error(
                # a list with the n-rows target values for the n-th step ahead
                [row.targets[n] for _, row in predictions_detail_df.iterrows()],
                # a list with the n-rows prediction values for the n-th step ahead
                [row.predictions[n] for _, row in predictions_detail_df.iterrows()]
            ) for n in targets_range
        ]
        predictions_summary_df['mae_vector'] = [mae_vector]

        # for index, row in dataframe.iterrows()
        rmse_vector = [
            sqrt(mean_squared_error(
                # a list with the n-rows target values for the n-th step ahead
                [row.targets[n] for _, row in predictions_detail_df.iterrows()],
                # a list with the n-rows prediction values for the n-th step ahead
                [row.predictions[n] for _, row in predictions_detail_df.iterrows()]
            )) for n in targets_range
        ]
        predictions_summary_df['rmse_vector'] = [rmse_vector]

        # for index, row in dataframe.iterrows()
        smape_vector = [
            smape(
                [row.targets[n] for _, row in predictions_detail_df.iterrows()],
                [row.predictions[n] for _, row in predictions_detail_df.iterrows()]
            ) for n in targets_range
        ]
        predictions_summary_df['smape_vector'] = [smape_vector]

        
    # insert count of rows as a column value
    predictions_summary[customer_id].insert(4, 'count', len(dataset_row_indexes_list))
    
    
    # DO NOT PERSIST THE PREDICTIONS DETAIL DATAFRAME
    persist_detail = False
    if persist_detail:
        # build a path to persist the dataframe to database/predictions_detail/
        detail_pickle_path = '{}/{}/{}/{}_{:02d}_{}_{}.pkl'.format(
            PROJECT_ROOT,
            'database',
            'predictions_detail',
            model_id,
            execution,
            dataset,
            inference)

        # persist the Pandas dataframe to database/predictions_detail/
        predictions_detail_df.to_pickle(detail_pickle_path)
        print('Persisted Pandas dataframe for predictions detail of {}_{:02d}_{}_{}'.format(model_id,
                                                                                            execution,
                                                                                            dataset,
                                                                                            inference))

    # DO NOT PERSIST THE PREDICTIONS SUMMARY DATAFRAME
    persist_summary = False
    if persist_summary:
        # build a path to persist the dataframe to database/predictions_summary/
        summary_pickle_path = '{}/{}/{}/{}_{:02d}_{}_{}.pkl'.format(
            PROJECT_ROOT,
            'database',
            'predictions_summary',
            model_id,
            execution,
            dataset,
            inference)

        # persist the Pandas dataframe to database/predictions_summary/
        predictions_summary_df.to_pickle(summary_pickle_path)
        print('Persisted Pandas dataframe for predictions summary of {}_{:02d}_{}_{}'.format(model_id,
                                                                                             execution,
                                                                                             dataset,
                                                                                             inference))


Exported model path is /home/developer/gcp/cbidmltsf/models/BSCTRFM_TPU_014_04/export/exporter/1631533140


In [75]:
predictions_detail[customer_id]

,model_id,execution,dataset,inference,customer_id,string_timestamps,predictions,targets,mae,nd,rmse,nrmse,smape
0,BSCTRFM_TPU_014,4,test,024,MT_321,"[2014-09-01 00:00:00, 2014-09-01 01:00:00, 201...","[88.3076171875, 78.01968383789062, 76.50577545...","[82.37394957983194, 76.06569900687546, 76.2547...",8.850989,0.073609,11.166868,0.092870,0.065336
1,BSCTRFM_TPU_014,4,test,024,MT_321,"[2014-09-01 01:00:00, 2014-09-01 02:00:00, 201...","[75.10681915283203, 69.64851379394531, 67.7572...","[76.06569900687546, 76.25477463712757, 69.7593...",8.105304,0.067372,10.486399,0.087164,0.065640
2,BSCTRFM_TPU_014,4,test,024,MT_321,"[2014-09-01 02:00:00, 2014-09-01 03:00:00, 201...","[71.50824737548828, 67.32380676269531, 63.4030...","[76.25477463712757, 69.75935828877004, 68.8006...",8.422953,0.069943,10.622921,0.088211,0.064843
3,BSCTRFM_TPU_014,4,test,024,MT_321,"[2014-09-01 03:00:00, 2014-09-01 04:00:00, 201...","[71.019775390625, 67.63922882080078, 63.755115...","[69.75935828877004, 68.80061115355232, 68.6115...",9.375288,0.077841,12.232538,0.101564,0.069089
4,BSCTRFM_TPU_014,4,test,024,MT_321,"[2014-09-01 04:00:00, 2014-09-01 05:00:00, 201...","[63.527626037597656, 61.554813385009766, 73.33...","[68.80061115355232, 68.61153552330026, 77.5954...",10.847628,0.090065,15.331624,0.127295,0.078591
...,...,...,...,...,...,...,...,...,...,...,...,...,...
140,BSCTRFM_TPU_014,4,test,024,MT_321,"[2014-09-06 20:00:00, 2014-09-06 21:00:00, 201...","[137.6163330078125, 122.4196548461914, 98.6898...","[136.74560733384274, 137.320473644003, 111.495...",13.294221,0.103773,19.254165,0.150296,0.098566
141,BSCTRFM_TPU_014,4,test,024,MT_321,"[2014-09-06 21:00:00, 2014-09-06 22:00:00, 201...","[117.67369842529297, 95.13550567626953, 79.293...","[137.320473644003, 111.4954163483576, 91.92704...",13.257244,0.103356,18.850493,0.146962,0.098271
142,BSCTRFM_TPU_014,4,test,024,MT_321,"[2014-09-06 22:00:00, 2014-09-06 23:00:00, 201...","[101.82263946533203, 85.4152603149414, 82.7541...","[111.4954163483576, 91.92704354469063, 84.2838...",12.877235,0.100368,18.847192,0.146899,0.092586
143,BSCTRFM_TPU_014,4,test,024,MT_321,"[2014-09-06 23:00:00, 2014-09-07 00:00:00, 201...","[88.86087799072266, 84.66204833984375, 79.7032...","[91.92704354469063, 84.28380443086326, 79.5072...",12.642913,0.098375,19.486085,0.151622,0.091951


In [76]:
predictions_summary[customer_id]

,model_id,execution,dataset,inference,count,customer_id,mae,nd,rmse,nrmse,smape
0,BSCTRFM_TPU_014,4,test,024,145,MT_321,7.749949,0.063973,10.79609,0.089021,0.061606


In [77]:
# plot metrics for the set of predictions

In [78]:
p = figure(title='MAE and RMSE over the test set for {}'.format(customer_id),
           width=960,
           height=320,
           x_axis_label='Test Dataset Row Index',
           y_axis_label='Error')

p.line(x=predictions_detail[customer_id].index,
       y=predictions_detail[customer_id]['mae'],
       legend_label='MAE',
       color='orange',
       line_width=1)

p.line(x=predictions_detail[customer_id].index,
       y=predictions_detail[customer_id]['rmse'],
       legend_label='RMSE',
       color='green',
       line_width=1)

show(p)

In [79]:
p = figure(title='ND, NRMSE, and SMAPE over the test set for {}'.format(customer_id),
           width=960,
           height=320,
           x_axis_label='Test Dataset Row Index',
           y_axis_label='Error')

p.line(x=predictions_detail[customer_id].index,
       y=predictions_detail[customer_id]['nd'],
       legend_label='ND',
       color='orange',
       line_width=2)

p.line(x=predictions_detail[customer_id].index,
       y=predictions_detail[customer_id]['nrmse'],
       legend_label='NRMSE',
       color='green',
       line_width=1)

p.line(x=predictions_detail[customer_id].index,
       y=predictions_detail[customer_id]['smape'],
       legend_label='SMAPE',
       color='purple',
       line_width=1)

show(p)

In [80]:
# now get a global, unique value for ND and NRSME for the 7 days in the test dataset

In [81]:
global_df = pd.DataFrame(columns=['prediction', 'target'])

In [82]:
# starting timestamps to predict over a complete day
start_timestamps = [0, 24, 48, 72, 96, 120, 144]

In [83]:
for start_timestamp in start_timestamps:
    # make a buffer dataframe for one-day predictions
    buffer_df = pd.DataFrame()
    # populate the buffer dataframe
    buffer_df['timestamp'] = pd.to_datetime(
        predictions_detail[customer_id]['string_timestamps'][start_timestamp:start_timestamp+1][start_timestamp]
    )
    buffer_df['prediction'] = \
        predictions_detail[customer_id]['predictions'][start_timestamp:start_timestamp+1][start_timestamp]
    
    buffer_df['target'] = \
        predictions_detail[customer_id]['targets'][start_timestamp:start_timestamp+1][start_timestamp]
    
    buffer_df = buffer_df.set_index('timestamp')
    
    global_df = pd.concat([global_df, buffer_df])

In [84]:
global_df

,prediction,target
2014-09-01 00:00:00,88.307617,82.373950
2014-09-01 01:00:00,78.019684,76.065699
2014-09-01 02:00:00,76.505775,76.254775
2014-09-01 03:00:00,71.705681,69.759358
2014-09-01 04:00:00,65.189095,68.800611
...,...,...
2014-09-07 19:00:00,144.437561,182.328113
2014-09-07 20:00:00,138.360474,140.574866
2014-09-07 21:00:00,130.450760,138.086325
2014-09-07 22:00:00,110.668182,116.720779


In [85]:
p = figure(title='Predictions over the test dataset for {}'.format(customer_id),
           width=960,
           height=320,
           x_axis_type='datetime',
           x_axis_label='Timestmap',
           y_axis_label='Value')

p.line(x=global_df.index,
       y=global_df['prediction'],
       legend_label='predictions',
       color='red',
       line_width=1)

p.line(x=global_df.index,
       y=global_df['target'],
       legend_label='targets',
       color='blue',
       line_width=1)

show(p)

### over the 7 days of the test dataset, daily forecasting windows, starting at midnight (2014-09-0X 00:00:00)

In [86]:
# get a global value for MAE
global_mae = mean_absolute_error(global_df['prediction'], global_df['target'])
global_mae

7.6293243243377145

In [87]:
# get a global average for true values
global_true_values_average = np.mean(global_df['target'])
global_true_values_average

120.95444996180292

In [88]:
# get a global value for ND
global_nd = global_mae/global_true_values_average
global_nd

0.06307601189329565

In [89]:
# get a global value for RMSE
global_rmse = sqrt(mean_squared_error(global_df['prediction'], global_df['target']))
global_rmse

11.993701951898492

In [90]:
# get a global value for NRMSE
global_nrmse = global_rmse/global_true_values_average
global_nrmse

0.09915883173943638

In [91]:
# get a global vale for SMAPE
global_smape = smape(global_df['prediction'], global_df['target'])
global_smape

0.05899791509145672